# Setup OpenAI API on Python

In [ ]:
! pip install openai
! pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.7 MB/s eta 0:00:00


#### 0. import libraries and set API key

In [4]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv("api.env")) # read local .env file

openai.organization = os.environ['ORGANIZATION']
openai.api_key  = os.environ['OPENAI_API_KEY']
#openai.api_key  = "sk-..."

#### 1. try the new connection with gpt-3.5

In [6]:
respone = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages = [
        {"role": "user", "content": "What is the capital of France?"}
    ],
)

print(respone)

{
  "id": "chatcmpl-7uu0xfiGt24er4pLWpMYm80Dkm6PU",
  "object": "chat.completion",
  "created": 1693796043,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The capital of France is Paris."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 14,
    "completion_tokens": 7,
    "total_tokens": 21
  }
}


# Understanding Prompt, Completion, Tokens

#### 1. Prompt

In [7]:
promp_EN = "What is the capital of Thailand?"
promp_TH = "ประเทศไทยมีเมืองหลวงชื่อว่าอะไร"

#### 2. Completion and Tokens

In [8]:
def get_completion_and_token_count(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )

    token_dict = {
        'prompt_tokens':response['usage']['prompt_tokens'],
        'completion_tokens':response['usage']['completion_tokens'],
        'total_tokens':response['usage']['total_tokens'],
    }
    return response.choices[0].message["content"], token_dict

#### 3. Tokens output and response

In [9]:
response, token_dict = get_completion_and_token_count(promp_EN)
print("response :",response)
print("token :", token_dict)

response : The capital of Thailand is Bangkok.
token : {'prompt_tokens': 14, 'completion_tokens': 7, 'total_tokens': 21}


In [10]:
response, token_dict = get_completion_and_token_count(promp_TH)
print("response :",response)
print("token :", token_dict)

response : เมืองหลวงของประเทศไทยชื่อว่ากรุงเทพมหานคร
token : {'prompt_tokens': 36, 'completion_tokens': 38, 'total_tokens': 74}


# Hierarchical text classification

In [11]:
def get_completion_and_token_count(messages,
                                   model="gpt-3.5-turbo",
                                   temperature=0,
                                   max_tokens=500):

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )

    content = response.choices[0].message["content"]

    token_dict = {
        'prompt_tokens':response['usage']['prompt_tokens'],
        'completion_tokens':response['usage']['completion_tokens'],
        'total_tokens':response['usage']['total_tokens'],
    }
    return content, token_dict

#### 1.System Message [1]

In [12]:
delimiter = "####"
system_message = f"""
You are a customer service assistant at an automotive company. \
You will be provided with customer service queries. \
The customer service query will be delimited with {delimiter} characters.
Hierarchical Classify each query into a primary category and a secondary category.
All secondary categories must belong to primary categories.
You will get short expainations for each categories inside &&& characters. \
Importantly, all the category you answer must strictly satisfy the short expainations if given.\
Provide your output in json format with the keys: primary and secondary. \

Primary categories:
- Shopping
- Owners
- Vehicles
- General Inquiry.

if "Shopping" is a primary category, secondary categories must follow:
- Dealership
- Compare Vehicles
- Financial Services and Payment
- Fits My Budget
- Buy Parts and Accessories

if "Owners" is a primary category, secondary categories must follow:
- Manuals and Warranties
- Schedule Service
- Saftety Recalls
- Technical Inquiry

if "Vehicles" is a primary category, secondary categories must follow:
- Cars and Minivan
- Trucks
- Crossovers and SUVs
- Electrified

if "General Inquiry" is a primary category, secondary categories must follow:
- Product information
- Pricing
- Feedback
- Speak to a human

&&&
Schedule Service is only for repair and maintenance service and not for a dealership
&&&

"""

In [13]:
def answer_queries(user_message, system_message):
    messages =  [
    {'role':'system',
     'content': system_message},
    {'role':'user',
     'content': f"{delimiter}{user_message}{delimiter}"},
    ]
    response, token_dict = get_completion_and_token_count(messages)
    print("user query :", user_message)
    print("output :", response)
    print("token :", token_dict)

user_messages = [
    "Can I schedule a dealership appointment online?",
    "How can I view my ToyotaCare benefits for my vehicle?",
    "Can a supercharger be installed on my vehicle?",
    "I want to seek information about the sedan. Do you have any suggestions?",
    "Can you suggest any SUV that costs no more than 2 million baht?",
    "Can Toyota Service Care be repurchased?",
    "How can I locate my nearest Toyota dealership?",
    "How do I make a payment to Toyota?",
    "How do I obtain an older model year brochure for a Toyota vehicle?",
    "I would like to add an aftermarket accessory or part to my vehicle. Will this impact my warranty?",
    "I would like to add an accessory or part to my vehicle.",
    """I purchased a part from an authorized Toyota dealership and installed it myself.\
    What warranty coverage do I have for this part?""",
    "Can I purchase a part from an authorized Toyota dealership and installe it myself",
]
i = 1
for user_message in user_messages:
    print(f"query: {i}")
    answer_queries(user_message, system_message)
    print("-------------------------------------------------------------------")
    i+=1

query: 1
user query : Can I schedule a dealership appointment online?
output : {
  "primary": "Shopping",
  "secondary": "Dealership"
}
token : {'prompt_tokens': 305, 'completion_tokens': 18, 'total_tokens': 323}
-------------------------------------------------------------------
query: 2
user query : How can I view my ToyotaCare benefits for my vehicle?
output : {
  "primary": "Owners",
  "secondary": "Manuals and Warranties"
}
token : {'prompt_tokens': 309, 'completion_tokens': 21, 'total_tokens': 330}
-------------------------------------------------------------------
query: 3
user query : Can a supercharger be installed on my vehicle?
output : {
  "primary": "Vehicles",
  "secondary": "Technical Inquiry"
}
token : {'prompt_tokens': 307, 'completion_tokens': 18, 'total_tokens': 325}
-------------------------------------------------------------------
query: 4
user query : I want to seek information about the sedan. Do you have any suggestions?
output : {
  "primary": "Vehicles",
  "s

#### 2.System Message [2]
- The result is the same as System Message [1].

In [14]:
delimiter = "####"
system_message = f"""
You are a customer service assistant at an automotive company. \
You will be provided with customer service queries. \
The customer service query will be delimited with {delimiter} characters.
Hierarchical Classify each query into a primary category and a secondary category.
All secondary categories must belong to primary categories.
You will get short expainations for each categories inside &&& characters. \
Importantly, all the category you answer must strictly satisfy the short expainations if given.\
Provide your output in json format with the keys: primary and secondary. \

Primary categories and secondary categories follow this format:
- primary category
    - secondary category
    - secondary category
    ...
- primary category
    - secondary category
    - secondary category
    ...
...

Categories:
- Shopping
    - Dealership
    - Compare Vehicles
    - Financial Services and Payment
    - Fits My Budget
    - Buy Parts and Accessories
- Owners
    - Manuals and Warranties
    - Schedule Service
    - Saftety Recalls
    - Technical Inquiry
- Vehicles
    - Cars and Minivan
    - Trucks
    - Crossovers and SUVs
    - Electrified
- General Inquiry.
    - Product information
    - Pricing
    - Feedback
    - Speak to a human

&&&
Schedule Service is only for repair and maintenance service and not for a dealership
&&&

"""

In [15]:
def answer_queries(user_message, system_message):
    messages =  [
    {'role':'system',
     'content': system_message},
    {'role':'user',
     'content': f"{delimiter}{user_message}{delimiter}"},
    ]
    response, token_dict = get_completion_and_token_count(messages)
    print("user query :", user_message)
    print("output :", response)
    print("token :", token_dict)

user_messages = [
    "Can I schedule a dealership appointment online?",
    "How can I view my ToyotaCare benefits for my vehicle?",
    "Can a supercharger be installed on my vehicle?",
    "I want to seek information about the sedan. Do you have any suggestions?",
    "Can you suggest any SUV that costs no more than 2 million baht?",
    "Can Toyota Service Care be repurchased?",
    "How can I locate my nearest Toyota dealership?",
    "How do I make a payment to Toyota?",
    "How do I obtain an older model year brochure for a Toyota vehicle?",
    "I would like to add an aftermarket accessory or part to my vehicle. Will this impact my warranty?",
    "I would like to add an accessory or part to my vehicle.",
    """I purchased a part from an authorized Toyota dealership and installed it myself.\
    What warranty coverage do I have for this part?""",
    "Can I purchase a part from an authorized Toyota dealership and installe it myself",
]
i = 1
for user_message in user_messages:
    print(f"query: {i}")
    answer_queries(user_message, system_message)
    print("-------------------------------------------------------------------")
    i+=1

query: 1
user query : Can I schedule a dealership appointment online?
output : {
  "primary": "Shopping",
  "secondary": "Dealership"
}
token : {'prompt_tokens': 305, 'completion_tokens': 18, 'total_tokens': 323}
-------------------------------------------------------------------
query: 2
user query : How can I view my ToyotaCare benefits for my vehicle?
output : {
  "primary": "Owners",
  "secondary": "Manuals and Warranties"
}
token : {'prompt_tokens': 309, 'completion_tokens': 21, 'total_tokens': 330}
-------------------------------------------------------------------
query: 3
user query : Can a supercharger be installed on my vehicle?
output : {
  "primary": "Vehicles",
  "secondary": "Technical Inquiry"
}
token : {'prompt_tokens': 307, 'completion_tokens': 18, 'total_tokens': 325}
-------------------------------------------------------------------
query: 4
user query : I want to seek information about the sedan. Do you have any suggestions?
output : {
  "primary": "Vehicles",
  "s

#### 3.System Message [3]
- The result is the similar as System Message [1] and [2].

In [16]:
delimiter = "####"
system_message = f"""
You are a customer service assistant at an automotive company. \
You will be provided with customer service queries. \
The customer service query will be delimited with {delimiter} characters.
Classify each query into a primary category and a secondary category as following:
    Step1: Classify a secondary category
    Step2: Determine a primary category using a secondary category
    Step3: Provide your output in json format with the keys: primary and secondary

Classification Note:
You will get short expainations for each categories inside &&& characters. \
Importantly, all the category you answer must strictly satisfy the short expainations if given.\
In your output, all secondary categories must strictly in primary categories.

Primary categories and secondary categories follow this format:
1 primary category
    1.1 secondary category
    1.2 secondary category
    ...
2 primary category
    2.1 secondary category
    2.2 secondary category
    ...
...

Categories:
1 Shopping
    1.1 Dealership
    1.2 Compare Vehicles
    1.3 Financial Services and Payment
    1.4 Fits My Budget
    1.5 Buy Parts and Accessories
2 Owners
    2.1 Manuals and Warranties
    2.2 Schedule Service
    2.3 Saftety Recalls
    2.4 Technical Inquiry
3 Vehicles
    3.1 Cars and Minivan
    3.2 Trucks
    3.3 Crossovers and SUVs
    3.4 Electrified
4 General Inquiry.
    4.1 Feedback
    4.2 Speak to a human

&&&
Schedule Service is only for repair and maintenance service and not for a dealership.
Vehicles is focus only in product information, specification and price.
&&&

"""

In [17]:
def answer_queries(user_message, system_message):
    messages =  [
    {'role':'system',
     'content': system_message},
    {'role':'user',
     'content': f"{delimiter}{user_message}{delimiter}"},
    ]
    response, token_dict = get_completion_and_token_count(messages)
    print("user query :", user_message)
    print("output :", response)
    print("token :", token_dict)

user_messages = [
    "Can I schedule a dealership appointment online?",
    "How can I view my ToyotaCare benefits for my vehicle?",
    "Can a supercharger be installed on my vehicle?",
    "I want to seek information about the sedan. Do you have any suggestions?",
    "Can you suggest any SUV that costs no more than 2 million baht?",
    "Can Toyota Service Care be repurchased?",
    "How can I locate my nearest Toyota dealership?",
    "How do I make a payment to Toyota?",
    "How do I obtain an older model year brochure for a Toyota vehicle?",
    "I would like to add an aftermarket accessory or part to my vehicle. Will this impact my warranty?",
    "I would like to add an accessory or part to my vehicle.",
    """I purchased a part from an authorized Toyota dealership and installed it myself.\
    What warranty coverage do I have for this part?""",
    "Can I purchase a part from an authorized Toyota dealership and installe it myself",
]
i = 1
for user_message in user_messages:
    print(f"query: {i}")
    answer_queries(user_message, system_message)
    print("-------------------------------------------------------------------")
    i+=1

query: 1
user query : Can I schedule a dealership appointment online?
output : {"primary": "Shopping", "secondary": "Dealership"}
token : {'prompt_tokens': 398, 'completion_tokens': 14, 'total_tokens': 412}
-------------------------------------------------------------------
query: 2
user query : How can I view my ToyotaCare benefits for my vehicle?
output : {"primary": "Owners", "secondary": "Manuals and Warranties"}
token : {'prompt_tokens': 402, 'completion_tokens': 17, 'total_tokens': 419}
-------------------------------------------------------------------
query: 3
user query : Can a supercharger be installed on my vehicle?
output : {"primary": "Vehicles", "secondary": "Technical Inquiry"}
token : {'prompt_tokens': 400, 'completion_tokens': 14, 'total_tokens': 414}
-------------------------------------------------------------------
query: 4
user query : I want to seek information about the sedan. Do you have any suggestions?
output : {"primary": "Vehicles", "secondary": "3.1 Cars an

# text classification - bottom to top

In [18]:
delimiter = "####"
system_message = f"""
You are a customer service assistant at an automotive company. \
You will be provided with customer service queries. \
The customer service query will be delimited with {delimiter} characters.
Perform as following:
    Step1: Classify a secondary category.
    Step2: Determine a primary category using a secondary category.
    Step3: Provide your output in json format with the keys: primary and secondary.

Classification Note:
You will get short expainations for each categories inside &&& characters. \
Importantly, all the category you answer must strictly satisfy the short expainations if given.\

Secondary categories:
- Dealership
- Compare Vehicles
- Financial Services and Payment
- Fits My Budget
- Buy Parts and Accessories
- Manuals and Warranties
- Schedule Service
- Saftety Recalls
- Technical Inquiry
- Cars and Minivan
- Trucks
- Crossovers and SUVs
- Electrified
- Feedback
- Speak to a human

Primary categories:
- Shopping if Secondary category is "Dealership", "Compare Vehicles", \
"Financial Services and Payment", "Fits My Budget" or "Buy Parts and Accessories".
- Owners if Secondary category is "Manuals and Warranties", "Schedule Service", \
"Saftety Recalls" or "Technical Inquiry".
- Vehicles if Secondary category is "Cars and Minivan", "Trucks", \
"Crossovers and SUVs" or "Electrified".
- General Inquiry if Secondary category is "Feedback" or "Speak to a human"

&&&
Schedule Service is only for repair and maintenance service and not for a dealership.
Vehicles is focus only in product information, specification and price.
Fits My Budget is a program that can suggest products given a customer's budget.
&&&

"""

In [19]:
def answer_queries(user_message, system_message):
    messages =  [
    {'role':'system',
     'content': system_message},
    {'role':'user',
     'content': f"{delimiter}{user_message}{delimiter}"},
    ]
    response, token_dict = get_completion_and_token_count(messages)
    print("user query :", user_message)
    print("output :", response)
    print("token :", token_dict)

user_messages = [
    "Can I schedule a dealership appointment online?",
    "How can I view my ToyotaCare benefits for my vehicle?",
    "Can a supercharger be installed on my vehicle?",
    "I want to seek information about the sedan. Do you have any suggestions?",
    "Can you suggest any SUV that costs no more than 2 million baht?",
    "Can you suggest any car that costs no more than 2 million baht?",
    "Can Toyota Service Care be repurchased?",
    "How can I locate my nearest Toyota dealership?",
    "How do I make a payment to Toyota?",
    "How do I obtain an older model year brochure for a Toyota vehicle?",
    "I would like to add an aftermarket accessory or part to my vehicle. Will this impact my warranty?",
    "I would like to add an accessory or part to my vehicle.",
    """I purchased a part from an authorized Toyota dealership and installed it myself.\
    What warranty coverage do I have for this part?""",
    "Can I purchase a part from an authorized Toyota dealership and installe it myself",
]
i = 1
for user_message in user_messages:
    print(f"query: {i}")
    answer_queries(user_message, system_message)
    print("-------------------------------------------------------------------")
    i+=1

query: 1
user query : Can I schedule a dealership appointment online?
output : {"primary": "Shopping", "secondary": "Dealership"}
token : {'prompt_tokens': 380, 'completion_tokens': 14, 'total_tokens': 394}
-------------------------------------------------------------------
query: 2
user query : How can I view my ToyotaCare benefits for my vehicle?
output : {"primary": "Owners", "secondary": "Manuals and Warranties"}
token : {'prompt_tokens': 384, 'completion_tokens': 17, 'total_tokens': 401}
-------------------------------------------------------------------
query: 3
user query : Can a supercharger be installed on my vehicle?
output : {"primary": "Vehicles", "secondary": "Technical Inquiry"}
token : {'prompt_tokens': 382, 'completion_tokens': 14, 'total_tokens': 396}
-------------------------------------------------------------------
query: 4
user query : I want to seek information about the sedan. Do you have any suggestions?
output : {"primary": "Vehicles", "secondary": "Cars and Mi

# Moderation

In [20]:
response = openai.Moderation.create(
    input="""
Here's the plan.  We get the warhead,
and we hold the world ransom...
...FOR ONE MILLION DOLLARS!
"""
)
moderation_output = response["results"][0]
print(moderation_output)

{
  "flagged": false,
  "categories": {
    "sexual": false,
    "hate": false,
    "harassment": false,
    "self-harm": false,
    "sexual/minors": false,
    "hate/threatening": false,
    "violence/graphic": false,
    "self-harm/intent": false,
    "self-harm/instructions": false,
    "harassment/threatening": false,
    "violence": false
  },
  "category_scores": {
    "sexual": 2.1709982e-06,
    "hate": 8.909444e-05,
    "harassment": 0.0018278483,
    "self-harm": 6.8723295e-07,
    "sexual/minors": 9.15072e-08,
    "hate/threatening": 5.7067416e-05,
    "violence/graphic": 3.7988924e-05,
    "self-harm/intent": 3.4265513e-06,
    "self-harm/instructions": 2.0105861e-07,
    "harassment/threatening": 0.002194881,
    "violence": 0.204964
  }
}


# Chain-of-Thought Prompting

In [21]:
delimiter = "####"
system_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags,\
i.e. {delimiter}.

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific product or products. \
Product cateogry doesn't count.

Step 2:{delimiter} If the user is asking about \
specific products, identify whether \
the products are in the following list.
All available products:
1. Product: Toyota Crown
   Category: Cars
   Brand: Toyota
   Model Number: 2023 TOYOTA CROWN XLE
   Basic Warranty: 36 months or 36,000 miles
   Features: Horsepower 236 hp, 2.4-Liter, 4-Cylinder DOHC 16-Valve, 5 Seating capacity
   Description: Toyota Crown is a distinctly crafted sedan unlike anything you've seen before.
   Price: $39,950.00

2. Product: Prius Prime
   Category: Cars
   Brand: Toyota
   Model Number: 2023 Prius Prime SE
   Basic Warranty: 36 months/36,000 miles
   Features: Horsepower 220 hp, 2.0-Liter, 4-cylinder DOHC 16-valve, 5 Seating capacity
   Description: Similarly to a standard Prius, but has an all-electric driving range
   Price: $32,350.00

3. Product: Highlander
   Category: SUVs
   Brand: Toyota
   Model Number: 2023 Highlander Hybrid LE
   Basic Warranty: 36 months/36,000 miles
   Features: Horsepower 243 hp, 2.5-Liter, 4-cylinder DOHC 16-valve, 5 Seating capacity
   Description: With advanced capabilities and a confident drive, Highlander puts you\
   and your crew in position to upgrade your adventures.
   Price: $42,220.00

4. Product: bZ4X
   Category: SUVs
   Brand: Toyota
   Model Number: 2023 bZ4X XLE
   Basic Warranty: 36 months/36,000 miles
   Features: Horsepower 201 hp, AC synchronous electric generator, 8 Seating capacity
   Description: an all-electric SUV that's available in two trim levels: XLE and Limited.
   Price: $42,000.00

5. Product: Tacoma
   Category: Trucks
   Brand: Toyota
   Model Number: 2023 Tacoma SR
   Basic Warranty: 36 months/36,000 miles
   Features: Horsepower 278 hp, 3.5-Liter, 6-cylinder direct-injection Atkinson-cycle engine \
   , 5 Seating capacity
   Description: The Tacoma is a mid-size pickup truck with a choice of two cabs and seating \
   for up to five
   Price: $34,515.00

Step 3:{delimiter} If the message contains products \
in the list above, list any assumptions that the \
user is making in their \
message e.g. that product X is bigger than \
product Y, or that product Z has a 36 months warranty.

Step 4:{delimiter}: If the user made any assumptions, \
figure out whether the assumption is true based on your \
product information.

Step 5:{delimiter}: First, politely correct the \
customer's incorrect assumptions if applicable. \
Only mention or reference products in the list of \
5 available products, as these are the only 5 \
products that the store sells. \
Answer the customer in a friendly tone.

Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.
"""

In [22]:
user_message = f"""
by how much is the 2023 Tacoma SR more expensive than the 2023 Highlander Hybrid LE"""

messages =  [
{'role':'system',
 'content': system_message},
{'role':'user',
 'content': f"{delimiter}{user_message}{delimiter}"},
]

response, token_dict = get_completion_and_token_count(messages)
print(response)
print(token_dict)

Step 1:#### The user is asking about the price difference between the 2023 Tacoma SR and the 2023 Highlander Hybrid LE.

Step 2:#### Both the 2023 Tacoma SR and the 2023 Highlander Hybrid LE are available products.

Step 3:#### The user assumes that the 2023 Tacoma SR is more expensive than the 2023 Highlander Hybrid LE.

Step 4:#### Based on the product information, the assumption is incorrect. The price of the 2023 Tacoma SR is $34,515.00, while the price of the 2023 Highlander Hybrid LE is $42,220.00. Therefore, the 2023 Highlander Hybrid LE is actually more expensive than the 2023 Tacoma SR.

Response to user:#### The 2023 Tacoma SR is actually less expensive than the 2023 Highlander Hybrid LE. The price of the 2023 Tacoma SR is $34,515.00, while the price of the 2023 Highlander Hybrid LE is $42,220.00.
{'prompt_tokens': 840, 'completion_tokens': 213, 'total_tokens': 1053}


In [23]:
user_message = f"""
by how much is the 2023 Highlander Hybrid LE more horsepower than the 2023 Prius Prime SE"""

messages =  [
{'role':'system',
 'content': system_message},
{'role':'user',
 'content': f"{delimiter}{user_message}{delimiter}"},
]

response, token_dict = get_completion_and_token_count(messages)
print(response)
print(token_dict)

Step 1:#### The user is asking a question about a specific product, comparing the horsepower of the 2023 Highlander Hybrid LE and the 2023 Prius Prime SE.

Step 2:#### Both the 2023 Highlander Hybrid LE and the 2023 Prius Prime SE are available products.

Step 3:#### The user assumes that the 2023 Highlander Hybrid LE has more horsepower than the 2023 Prius Prime SE.

Step 4:#### Based on the product information, the 2023 Highlander Hybrid LE has 243 horsepower, while the 2023 Prius Prime SE has 220 horsepower. Therefore, the 2023 Highlander Hybrid LE has 23 more horsepower than the 2023 Prius Prime SE.

Response to user:#### The 2023 Highlander Hybrid LE has 23 more horsepower than the 2023 Prius Prime SE.
{'prompt_tokens': 842, 'completion_tokens': 180, 'total_tokens': 1022}


In [24]:
user_message = f"""
I want to buy the 2023 Toyota Crown XLE, because it have the most horsepower, seat, and I love SUV.
"""
messages =  [
{'role':'system',
 'content': system_message},
{'role':'user',
 'content': f"{delimiter}{user_message}{delimiter}"},
]

response, token_dict = get_completion_and_token_count(messages)
print(response)
print(token_dict)

Step 1:#### The user is asking about a specific product, the 2023 Toyota Crown XLE.

Step 2:#### The 2023 Toyota Crown XLE is available for purchase.

Step 3:#### The user assumes that the 2023 Toyota Crown XLE has the most horsepower, seating capacity, and is an SUV.

Step 4:#### The assumption about the 2023 Toyota Crown XLE having the most horsepower and seating capacity is incorrect. The 2023 Toyota Crown XLE has a horsepower of 236 hp and a seating capacity of 5, which is less than some other models like the Highlander with 243 hp and the Tacoma with 278 hp.

Response to user:#### Thank you for your interest in the 2023 Toyota Crown XLE. While it is a great sedan with a horsepower of 236 hp and a seating capacity of 5, it is not an SUV. If you are looking for an SUV with more horsepower and seating capacity, I would recommend considering the Highlander or the Tacoma. Let me know if you have any other questions or if there's anything else I can assist you with.
{'prompt_tokens': 84

## Link the information with Chaining Prompts

In [25]:
# product information
products = {
    "Toyota Crown XLE": {
        "name": "Toyota Crown",
        "category": "Cars",
        "brand": "Toyota",
        "model_number": "2023 TOYOTA CROWN XLE",
        "warranty": "36 months or 36,000 miles",
        "features": ["Horsepower 236 hp", "2.4-Liter, 4-Cylinder DOHC 16-Valve", "5 Seating capacity"],
        "description": "Toyota Crown is a distinctly crafted sedan unlike anything you've seen before.",
        "price": 39950.00
    },
    "Prius Prime SE": {
        "name": "Prius Prime",
        "category": "Cars",
        "brand": "Toyota",
        "model_number": "2023 Prius Prime SE",
        "warranty": "36 months or 36,000 miles",
        "features": ["Horsepower 220 hp", "2.0-Liter, 4-Cylinder DOHC 16-Valve", "5 Seating capacity"],
        "description": "Similarly to a standard Prius, but has an all-electric driving range.",
        "price": 32350.00
    },
    "Highlander Hybrid LE": {
        "name": "Highlander Hybrid",
        "category": "SUVs",
        "brand": "Toyota",
        "model_number": "2023 Highlander Hybrid LE",
        "warranty": "36 months or 36,000 miles",
        "features": ["Horsepower 243 hp", "2.5-Liter, 4-Cylinder DOHC 16-Valve", "5 Seating capacity"],
        "description": "With advanced capabilities and a confident drive, Highlander puts you and your crew in position to upgrade your adventures.",
        "price": 42220.00
    },
    "bZ4X XLE": {
        "name": "bZ4X",
        "category": "SUVs",
        "brand": "Toyota",
        "model_number": "2023 bZ4X XLE",
        "warranty": "36 months or 36,000 miles",
        "features": ["Horsepower 201 hp", "AC synchronous electric generator", "8 Seating capacity"],
        "description": "an all-electric SUV that's available in two trim levels: XLE and Limited.",
        "price": 42000.00
    },
    "Tacoma SR": {
        "name": "Tacoma",
        "category": "Trucks",
        "brand": "Toyota",
        "model_number": "2023 Tacoma SR",
        "warranty": "36 months or 36,000 miles",
        "features": ["Horsepower 278 hp", "3.5-Liter, 6-cylinder direct-injection Atkinson-cycle engine", "5 Seating capacity"],
        "description": "The Tacoma is a mid-size pickup truck with a choice of two cabs and seating for up to five.",
        "price": 34515.00
    },
}

In [26]:
delimiter = "####"
system_message_1 = f"""
You will be provided with customer service queries. \
The customer service query will be delimited with \
{delimiter} characters.
Output a python list of objects, where each object has \
the following format:
    'category': <one of Computers and Laptops, \
    Smartphones and Accessories, \
    Televisions and Home Theater Systems, \
    Gaming Consoles and Accessories,
    Audio Equipment, Cameras and Camcorders>,
OR
    'products': <a list of products that must \
    be found in the allowed products below>

Where the categories and products must be found in \
the customer service query.
If a product is mentioned, it must be associated with \
the correct category in the allowed products list below.
If no products or categories are found, output an \
empty list.

Allowed products:

Cars category:
Toyota Crown
Prius Prime

SUVs category:
Highlander
bZ4X

Trucks category:
Tacoma

Only output the list of objects, with nothing else.
"""
system_message_2 = f"""
You are a customer service assistant for a automobile showroom. \
Respond in a friendly and helpful tone, \
with very concise answers. \
Make sure to ask the user relevant follow up questions.
"""
user_message = f"""tell me about the Toyota Crown XLE and the Highlander Hybrid LE. What is different?"""

In [27]:
import json

def get_product_by_name(name):
    return products.get(name, None)

def get_products_by_category(category):
    return [product for product in products.values() if product["category"] == category]

def read_string_to_list(input_string):
    if input_string is None:
        return None

    try:
        input_string = input_string.replace("'", "\"")  # Replace single quotes with double quotes for valid JSON
        data = json.loads(input_string)
        return data
    except json.JSONDecodeError:
        print("Error: Invalid JSON string")
        return None

def generate_output_string(data_list):
    output_string = ""

    if data_list is None:
        return output_string

    for data in data_list:
        try:
            if "products" in data:
                products_list = data["products"]
                for product_name in products_list:
                    product = get_product_by_name(product_name)
                    if product:
                        output_string += json.dumps(product, indent=4) + "\n"
                    else:
                        print(f"Error: Product '{product_name}' not found")
            elif "category" in data:
                category_name = data["category"]
                category_products = get_products_by_category(category_name)
                for product in category_products:
                    output_string += json.dumps(product, indent=4) + "\n"
            else:
                print("Error: Invalid object format")
        except Exception as e:
            print(f"Error: {e}")

    return output_string

In [28]:
messages =  [
{'role':'system',
 'content': system_message_1},
{'role':'user',
 'content': f"{delimiter}{user_message}{delimiter}"},
]
category_and_product_response, token_dict_1 = get_completion_and_token_count(messages)
category_and_product_list = read_string_to_list(category_and_product_response)
product_information_for_user_message = generate_output_string(category_and_product_list)

messages =  [
{'role':'system',
 'content': system_message_2},
{'role':'user',
 'content': user_message},
{'role':'assistant',
 'content': f"""Relevant product information:\n\
 {product_information_for_user_message}"""},
]
final_response, token_dict_2 = get_completion_and_token_count(messages)
print(final_response)
print(token_dict_1)
print(token_dict_2)

The Toyota Crown XLE is a sedan with a 2.4-Liter, 4-Cylinder engine and a seating capacity of 5. It has a horsepower of 236 hp and comes with a warranty of 36 months or 36,000 miles. The Highlander Hybrid LE, on the other hand, is an SUV with a 2.5-Liter, 4-Cylinder engine and a seating capacity of 5. It has a horsepower of 243 hp and also comes with a warranty of 36 months or 36,000 miles. Is there anything specific you would like to know about these models?
{'prompt_tokens': 224, 'completion_tokens': 26, 'total_tokens': 250}
{'prompt_tokens': 334, 'completion_tokens': 127, 'total_tokens': 461}


# Check outputs

In [29]:
def get_completion_from_messages(messages,
                                 model="gpt-3.5-turbo",
                                 temperature=0,
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut
    )
    return response

In [30]:
final_response_to_customer = f"""
The Toyota Crown is a sedan while the Highlander is an SUV. \
The Highlander has a higher horsepower and seating capacity \
compared to the Crown. Is there anything else you would like \
to know about these vehicles?
"""
system_message = f"""
You are an assistant that evaluates whether \
customer service agent responses sufficiently \
answer customer questions, and also validates that \
all the facts the assistant cites from the product \
information are correct.
The product information and user and customer \
service agent messages will be delimited by \
3 backticks, i.e. ```.
Respond with a Y or N character, with no punctuation:
Y - if the output sufficiently answers the question \
AND the response correctly uses product information
N - otherwise

Output a single letter only.
"""
customer_message = f"""tell me about the Toyota Crown and Highlander. What is different?"""

In [31]:
category_and_product_target = """
[
{'category': 'Cars', 'products': ['Toyota Crown']},
{'category': 'SUVs', 'products': ['Highlander']}
]
"""
category_and_product_list_target = read_string_to_list(category_and_product_target)
product_information = generate_output_string(category_and_product_list_target)
q_a_pair = f"""
Customer message: ```{customer_message}```
Product information: ```{product_information}```
Agent response: ```{final_response_to_customer}```

Does the response use the retrieved information correctly?
Does the response sufficiently answer the question

Output Y or N
"""
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': q_a_pair}
]

response, token_dict = get_completion_and_token_count(messages)
print(response)

Error: Product 'Toyota Crown' not found
Error: Product 'Highlander' not found
Y


In [32]:
category_and_product_target = """
[
{'category': 'Trucks', 'products': ['Tacoma']}
]
"""
category_and_product_list_target = read_string_to_list(category_and_product_target)
product_information = generate_output_string(category_and_product_list_target)
q_a_pair = f"""
Customer message: ```{customer_message}```
Product information: ```{product_information}```
Agent response: ```{final_response_to_customer}```

Does the response use the retrieved information correctly?
Does the response sufficiently answer the question

Output Y or N
"""
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': q_a_pair}
]

response, token_dict = get_completion_and_token_count(messages)
print(response)

Error: Product 'Tacoma' not found
Y


In [33]:
category_and_product_target = """
[
{'category': 'Trucks', 'products': ['Tacoma']},
{'category': 'SUVs', 'products': ['Highlander']}
]
"""
category_and_product_list_target = read_string_to_list(category_and_product_target)
product_information = generate_output_string(category_and_product_list_target)
q_a_pair = f"""
Customer message: ```{customer_message}```
Product information: ```{product_information}```
Agent response: ```{final_response_to_customer}```

Does the response use the retrieved information correctly?
Does the response sufficiently answer the question

Output Y or N
"""
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': q_a_pair}
]

response, token_dict = get_completion_and_token_count(messages)
print(response)

Error: Product 'Tacoma' not found
Error: Product 'Highlander' not found
Y


In [34]:
category_and_product_target = """
[
{'category': 'SUVs', 'products': ['Highlander']}
]
"""
category_and_product_list_target = read_string_to_list(category_and_product_target)
product_information = generate_output_string(category_and_product_list_target)
q_a_pair = f"""
Customer message: ```{customer_message}```
Product information: ```{product_information}```
Agent response: ```{final_response_to_customer}```

Does the response use the retrieved information correctly?
Does the response sufficiently answer the question

Output Y or N
"""
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': q_a_pair}
]

response, token_dict = get_completion_and_token_count(messages)
print(response)

Error: Product 'Highlander' not found
Y


# Summarize text

In [35]:
text = f"""
Step 1:#### The user is asking about a specific product, Toyota Crown.
Step 2:#### Toyota Crown is a car, not an SUV. It has 236 horsepower and seats 5 people.
Step 3:#### The user assumes that Toyota Crown is an SUV and has the most horsepower and seats.
Step 4:#### Toyota Crown is not an SUV, it is a sedan. It has 236 horsepower and seats 5 people,\
which is not the most horsepower or seating capacity among the available products.
Response to user:#### Thank you for your interest in Toyota Crown. However, I would like to \
clarify that Toyota Crown is a sedan, not an SUV. It has 236 horsepower and seats 5 people. \
If you are interested in an SUV, we have Highlander and bZ4X available.
"""
prompt = f"""
Summarize the text delimited by triple backticks \
into a single sentence.
```{text}```
"""
messages = [
    {'role': 'user', 'content': prompt}
]
response, token_dict = get_completion_and_token_count(messages)
print(response)
print(token_dict)

The user is asking about the Toyota Crown, assuming it is an SUV with the most horsepower and seating capacity, but the response clarifies that it is actually a sedan with 236 horsepower and seating for 5 people, and suggests other SUV options available.
{'prompt_tokens': 195, 'completion_tokens': 51, 'total_tokens': 246}


# Step instructions to a Paragraph

In [36]:
text_1 = """
Step 1 - Loosen the front tires slightly. This should be done with a tire iron or impact wrench. While on the ground the car's weight will keep the wheels from turning. This way you can safely loosen the lug nuts.
Step 2 - Raise the front end with a floor jack. Refer to your service manual for jacking points and jack up your car. Stabilize the vehicle with jack stands and chock the rear tires. It is not safe to leave the vehicle suspended on the jack alone.
Step 3 - Remove the wheel. Finish taking the lug nuts off of the wheel and remove it from the wheelbase. Slide the wheel under the car. This provides an extra safety measure in case the jack stands fail.
Step 4 - Locate the outer tie rod end. When you remove the wheel, you will be able to see the steering knuckle. There will be a shaft going through this knuckle with a castle nut on the bottom and a round head on top. This is the outer tie rod.
Step 5 - Locate the inner tie rod end. Cars often have an inner tie rod end as well. Start by locating the outer tie rod end. Follow the outer tie rod end underneath the car until it meets the inner tie rod end.
"""
prompt = f"""
You will be provided with text delimited by triple quotes.
If the instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

re-write a sequence of instructions as single paragraph

\"\"\"{text_1}\"\"\"
"""
messages = [
    {'role': 'user', 'content': prompt}
]
response_1, token_dict = get_completion_and_token_count(messages)
print("Completion for Text 1:")
print(response_1)
print(token_dict)

Completion for Text 1:
To begin, loosen the front tires slightly using a tire iron or impact wrench while the car is on the ground to prevent the wheels from turning. This will allow you to safely loosen the lug nuts. Next, raise the front end of the car with a floor jack, referring to the service manual for the proper jacking points. Once the car is raised, stabilize it with jack stands and chock the rear tires to ensure it is secure. With the car elevated, remove the wheel by finishing taking off the lug nuts and sliding the wheel off the wheelbase. As an extra safety measure, slide the wheel under the car in case the jack stands fail. Moving on, locate the outer tie rod end by removing the wheel and observing the steering knuckle. Look for a shaft going through the knuckle with a castle nut on the bottom and a round head on top - this is the outer tie rod. Additionally, cars often have an inner tie rod end. To find it, start by locating the outer tie rod end and follow it underneath

# Summarize text to step instructions

In [37]:
text_2 = response_1
prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
messages = [
    {'role': 'user', 'content': prompt}
]
response, token_dict = get_completion_and_token_count(messages)
print("Completion for Text 1:")
print(response)
print(token_dict)

Completion for Text 1:
Step 1 - Loosen the front tires slightly using a tire iron or impact wrench while the car is on the ground to prevent the wheels from turning.
Step 2 - Raise the front end of the car with a floor jack, referring to the service manual for the proper jacking points.
Step 3 - Stabilize the car with jack stands and chock the rear tires to ensure it is secure.
Step 4 - Remove the wheel by finishing taking off the lug nuts and sliding the wheel off the wheelbase.
Step 5 - Slide the wheel under the car as an extra safety measure.
Step 6 - Locate the outer tie rod end by removing the wheel and observing the steering knuckle.
Step 7 - Look for a shaft going through the knuckle with a castle nut on the bottom and a round head on top - this is the outer tie rod.
Step 8 - Find the inner tie rod end by starting at the outer tie rod end and following it underneath the car until it meets the inner tie rod end.
{'prompt_tokens': 292, 'completion_tokens': 210, 'total_tokens': 502

# Few Short learning

In [38]:
prompt = f"""
Your task is to answer in a consistent style.

<Question>: A "whatpu" is a small, furry animal native to Tanzania.
An example of a sentence that uses the word whatpu is:

<Answer>: We were traveling in Africa and we saw these very cute whatpus.

<Question>: To do a "farduddle" means to jump up and down really fast.
An example of a sentence that uses the word farduddle is.
"""
messages = [
    {'role': 'user', 'content': prompt}
]
response, token_dict = get_completion_and_token_count(messages)
print(response)

<Answer>: The children were so excited that they started to farduddle in the playground.


In [39]:
prompt = f"""
Your task is to answer in a consistent style.

This is awesome! // Negative
This is bad! // Positive
Wow that movie was rad! // Positive
What a horrible show! //
"""
messages = [
    {'role': 'user', 'content': prompt}
]
response, token_dict = get_completion_and_token_count(messages)
print(response)

Positive


In [40]:
prompt = f"""
Your task is to determine the following five numbers\
of the sequence. And describe the logic of the given series.

1, 2, 3 ,5 ,8, 13, ...

"""
messages = [
    {'role': 'user', 'content': prompt}
]
response, token_dict = get_completion_and_token_count(messages)
print(response)

The logic of the given series is based on the Fibonacci sequence. 

In the Fibonacci sequence, each number is the sum of the two preceding ones. So, to determine the next number in the sequence, we add the last two numbers together.

Starting with 1 and 2, we add them to get 3. Then, we add 2 and 3 to get 5. Continuing this pattern, we add 3 and 5 to get 8, and so on.

Therefore, the next five numbers in the sequence would be:

21, 34, 55, 89, 144


In [41]:
prompt = f"""
Your task is to determine the following five numbers\
of the sequence. And describe the logic of the given series.

3, 3.1, 3.14, 3.141, 3.1415, ...

"""
messages = [
    {'role': 'user', 'content': prompt}
]
response, token_dict = get_completion_and_token_count(messages)
print(response)

The logic of the given series is that each number is an approximation of the mathematical constant π (pi). 

The first number in the series is 3, which is a whole number approximation of π. 

The second number in the series is 3.1, which is a decimal approximation of π rounded to one decimal place. 

The third number in the series is 3.14, which is a decimal approximation of π rounded to two decimal places. 

The fourth number in the series is 3.141, which is a decimal approximation of π rounded to three decimal places. 

The fifth number in the series is 3.1415, which is a decimal approximation of π rounded to four decimal places. 

In general, each number in the series is obtained by adding one more decimal place to the previous number, resulting in a more accurate approximation of π.


In [42]:
prompt = f"""
Your task is to determine the following 15 numbers\
of the sequence. And describe the logic of the given series.

11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, ...

"""
messages = [
    {'role': 'user', 'content': prompt}
]
response, token_dict = get_completion_and_token_count(messages)
print(response)

The given sequence consists of prime numbers starting from 11. 

To determine the next numbers in the sequence, we need to identify the pattern or logic behind the prime numbers. 

Upon observing the sequence, we can notice that all the numbers in the sequence are consecutive prime numbers. 

Prime numbers are numbers that are only divisible by 1 and themselves. 

Starting from 11, the next prime number is 13, followed by 17, 19, 23, and so on. 

Therefore, the next 15 numbers in the sequence are:

67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137


# Extract information

In [43]:
text = f"""
We are proud to announce a significant enhancement to our Hybrid Battery Warranty.

Starting with the 2020 model year, every Toyota hybrid battery warranty is being \
increased from 8 years or 100,000 miles, to 10 years from date of first use, or \
150,000 miles whichever comes first. This enhancement serves as an indication of \
our confidence in the quality, dependability, and reliability of our products.

On top of our basic coverage, we offer:

    - Hybrid-Related Component Coverage1

        Hybrid-related components, including the HV battery, \
        battery control module, hybrid control module and inverter \
        with converter, are covered for 8 years/100,000 miles. \
        The HV battery may have longer coverage under emissions warranty.\
        Refer to applicable Warranty & Maintenance Guide for details.

    - Powertrain Coverage1

        60 months/60,000 miles (engine, transmission/transaxle, \
        front-wheel-drive system, rear-wheel drive, seatbelts and airbags).

    - Rust-Through Coverage1

        60 months/unlimited miles (corrosion perforation of sheet metal).

    - Emissions Coverage1

        Coverages vary under Federal and California regulations. \
        Refer to applicable Warranty & Maintenance Guide for details.

For Toyota hybrid vehicles beginning with model year 2020, \
the hybrid (HV) battery is covered for 10 years from original \
date of first use or 150,000 miles, whichever comes first. \
Coverage is subject to the terms and conditions of your \
New Vehicle Limited Warranty. See Owner's Warranty and \
Maintenance Guide for details.
"""

In [44]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long.

Format your response as a list of items separated by commas.

Text sample: '''{text}'''
"""

messages = [
    {'role': 'user', 'content': prompt}
]

response, token_dict = get_completion_and_token_count(messages)
print(response)
print(token_dict)

1. Hybrid Battery Warranty
2. Toyota hybrid vehicles
3. Warranty coverage
4. Powertrain coverage
5. Rust-Through coverage
{'prompt_tokens': 378, 'completion_tokens': 29, 'total_tokens': 407}


In [45]:
prompt_1 = f"""
Perform the following actions:
Task 1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
Task 2 - Extract information for each component and their coverage range following this format:

OVERVIEW
-

COVERAGE LIST
- <component 1>
- <component 2>
...

<component 1>
-
-
...

<component 2>
-
-
...

...


Here this example:

OVERVIEW
- Every Toyota vehicle is supported by \
a 36-month/36,000-mile limited warranty coverage. \
But it doesn't stop there.

COVERAGE LIST
- XYZ
- Powertrain

XYZ
- ABC component: 36 months/36,000 miles
- DFG component: 36 months/36,000 miles
- HIJ component: 24 months/24,000 miles

Powertrain
- engine: 60 months/60,000 miles
- transmission: 60 months/60,000 miles

Text:
```{text}```

"""
messages = [
    {'role': 'user', 'content': prompt_1}
]
response, token_dict = get_completion_and_token_count(messages)
print(response)
print(token_dict)

Task 1 - The text describes an enhancement to Toyota's Hybrid Battery Warranty, increasing the coverage to 10 years or 150,000 miles for the 2020 model year.

Task 2 - 

OVERVIEW
- Every Toyota vehicle is supported by a 36-month/36,000-mile limited warranty coverage. But it doesn't stop there.

COVERAGE LIST
- Hybrid-Related Component Coverage
- Powertrain Coverage
- Rust-Through Coverage
- Emissions Coverage

Hybrid-Related Component Coverage
- HV battery: 8 years/100,000 miles
- Battery control module: 8 years/100,000 miles
- Hybrid control module: 8 years/100,000 miles
- Inverter with converter: 8 years/100,000 miles

Powertrain Coverage
- Engine: 60 months/60,000 miles
- Transmission/transaxle: 60 months/60,000 miles
- Front-wheel-drive system: 60 months/60,000 miles
- Rear-wheel drive: 60 months/60,000 miles
- Seatbelts: 60 months/60,000 miles
- Airbags: 60 months/60,000 miles

Rust-Through Coverage
- Corrosion perforation of sheet metal: 60 months/unlimited miles

Emissions Cover

In [46]:
fact_sheet_chair = """
OVERVIEW
- Toyota hybrid battery warranty increased to 10 years or 150,000 miles.
- Additional coverage includes hybrid-related component coverage, powertrain coverage\
, rust-through coverage, and emissions coverage.

COVERAGE LIST
- Hybrid-Related Component Coverage
- Powertrain Coverage
- Rust-Through Coverage
- Emissions Coverage

Hybrid-Related Component Coverage
- HV battery: 8 years/100,000 miles (may have longer coverage under emissions warranty)
- Battery control module: 8 years/100,000 miles
- Hybrid control module: 8 years/100,000 miles
- Inverter with converter: 8 years/100,000 miles

Powertrain Coverage
- Engine: 60 months/60,000 miles
- Transmission/transaxle: 60 months/60,000 miles
- Front-wheel-drive system: 60 months/60,000 miles
- Rear-wheel drive: 60 months/60,000 miles
- Seatbelts: 60 months/60,000 miles
- Airbags: 60 months/60,000 miles

Rust-Through Coverage
- 60 months/unlimited miles (corrosion perforation of sheet metal)

Emissions Coverage
- Coverages vary under Federal and California regulations. Refer to \
applicable Warranty & Maintenance Guide for details.
"""

In [47]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks. Then transalate in thai.

Technical specifications: ```{fact_sheet_chair}```
"""

messages = [
    {'role': 'user', 'content': prompt}
]

response, token_dict = get_completion_and_token_count(messages)
print(response)
print(token_dict)

Product Description (English):
Introducing the new Toyota hybrid with an extended battery warranty of 10 years or 150,000 miles. This means you can enjoy the benefits of a hybrid vehicle for even longer without worrying about battery issues. But that's not all - our warranty also covers other hybrid-related components, powertrain, rust-through, and emissions. 

With our Hybrid-Related Component Coverage, you can rest assured knowing that the HV battery, battery control module, hybrid control module, and inverter with converter are all covered for 8 years or 100,000 miles. This ensures the longevity and reliability of your hybrid system.

The Powertrain Coverage includes the engine, transmission/transaxle, front-wheel-drive system, rear-wheel drive, seatbelts, and airbags, all covered for 60 months or 60,000 miles. This comprehensive coverage gives you peace of mind knowing that the essential components of your vehicle are protected.

We also offer Rust-Through Coverage for 60 months wi

In [48]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

The description is intended for toyota dealership,
so should be technical in nature and focus on the
product warranty.

After the description, include a table that gives the
component's warranty duration. The table should have tree columns.
In the first column include the name of the component.
In the second column include the coverage group.
In the third column include the warranty duration in months and miles.

Give the table the title 'Product Warranty'.

Format everything as HTML that can be used in a website.
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

messages = [
    {'role': 'user', 'content': prompt}
]

response, token_dict = get_completion_and_token_count(messages)
print(response)
print(token_dict)

<div>
  <h2>Product Description</h2>
  <p>Introducing the latest Toyota hybrid model, equipped with cutting-edge technology and backed by an exceptional warranty. With our new and improved warranty, you can have peace of mind knowing that your investment is protected for years to come.</p>
  <p>One of the standout features of this Toyota hybrid is the extended hybrid battery warranty. We have increased the warranty duration to an impressive 10 years or 150,000 miles. This means that you can enjoy the benefits of hybrid technology without worrying about the battery's performance or longevity.</p>
  <p>In addition to the hybrid battery warranty, our coverage extends to various other components. The hybrid-related component coverage ensures that critical parts such as the battery control module, hybrid control module, and inverter with converter are protected for 8 years or 100,000 miles. This comprehensive coverage ensures that your hybrid system remains in top condition.</p>
  <p>Our po

In [49]:
from IPython.display import display, HTML
display(HTML(response))

Component,Coverage Group,Warranty Duration (Months/Miles)
HV battery,Hybrid-Related Component Coverage,"8 years/100,000 miles"
Battery control module,Hybrid-Related Component Coverage,"8 years/100,000 miles"


# mutiple task

In [50]:
text = f"""
We are proud to announce a significant enhancement to our Hybrid Battery Warranty.

Starting with the 2020 model year, every Toyota hybrid battery warranty is being \
increased from 8 years or 100,000 miles, to 10 years from date of first use, or \
150,000 miles whichever comes first. This enhancement serves as an indication of \
our confidence in the quality, dependability, and reliability of our products.

On top of our basic coverage, we offer:

    - Hybrid-Related Component Coverage1

        Hybrid-related components, including the HV battery, \
        battery control module, hybrid control module and inverter \
        with converter, are covered for 8 years/100,000 miles. \
        The HV battery may have longer coverage under emissions warranty.\
        Refer to applicable Warranty & Maintenance Guide for details.

    - Powertrain Coverage1

        60 months/60,000 miles (engine, transmission/transaxle, \
        front-wheel-drive system, rear-wheel drive, seatbelts and airbags).

    - Rust-Through Coverage1

        60 months/unlimited miles (corrosion perforation of sheet metal).

    - Emissions Coverage1

        Coverages vary under Federal and California regulations. \
        Refer to applicable Warranty & Maintenance Guide for details.

For Toyota hybrid vehicles beginning with model year 2020, \
the hybrid (HV) battery is covered for 10 years from original \
date of first use or 150,000 miles, whichever comes first. \
Coverage is subject to the terms and conditions of your \
New Vehicle Limited Warranty. See Owner's Warranty and \
Maintenance Guide for details.
"""
# example 1
prompt_1 = f"""
Perform the following actions:
Task 1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
Task 2 - Translate the summary into Thai.
Task 3 - Translate the summary into Japanese.
Task 4 - Determine whether the original is formal or informal.
Task 5 - If the original is formal, \transform the summary to \
informal; else, convert the summary to informal.

Text:
```{text}```

"""
messages = [
    {'role': 'user', 'content': prompt_1}
]
response, token_dict = get_completion_and_token_count(messages)
print(response)
print(token_dict)

Task 1 - The Toyota Hybrid Battery Warranty has been extended to 10 years or 150,000 miles for the 2020 model year, indicating confidence in the quality and reliability of their products.

Task 2 - การรับประกันแบตเตอรี่ไฮบริดของโตโยต้าได้ถูกขยายอายุเป็น 10 ปีหรือ 150,000 ไมล์สำหรับรุ่นปี 2020 เป็นการแสดงถึงความมั่นใจในคุณภาพและความเชื่อถือได้ของผลิตภัณฑ์ของพวกเขา

Task 3 - トヨタのハイブリッドバッテリー保証は、2020年モデルイヤーから10年または150,000マイルに延長され、製品の品質と信頼性に対する自信の表れとなっています。

Task 4 - The original is formal.

Task 5 - The summary does not need to be converted to informal as it is already informal.
{'prompt_tokens': 422, 'completion_tokens': 311, 'total_tokens': 733}


# respond email

In [51]:
# given the sentiment from the lesson on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a blender
review = f"""
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

In [52]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service.
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
messages = [
    {'role': 'user', 'content': prompt}
]
response, token_dict = get_completion_and_token_count(messages)
print(response)

Dear Valued Customer,

Thank you for taking the time to share your review with us. We appreciate your feedback and apologize for any inconvenience you may have experienced.

We are sorry to hear about the price increase you noticed in December. Our pricing is subject to change based on various factors, including market conditions and demand. However, we understand your frustration and apologize for any confusion caused.

Regarding the quality of the base, we appreciate your observation. We constantly strive to improve our products and your feedback will be taken into consideration for future enhancements.

We apologize for the issue you encountered with the motor after the warranty expired. Our customer service team is always available to assist you with any concerns you may have. Please feel free to reach out to them for further assistance.

Thank you once again for your valuable feedback. We truly value your loyalty and appreciate your support. If you have any further questions or ne

In [53]:
# given the sentiment from the lesson on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a blender
review = f"""
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

In [54]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service.
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
messages = [
    {'role': 'user', 'content': prompt}
]
response, token_dict = get_completion_and_token_count(messages)
print(response)

Dear Valued Customer,

Thank you for taking the time to share your review with us. We appreciate your feedback and apologize for any inconvenience you may have experienced.

We are sorry to hear about the price increase you noticed in December. Our pricing is subject to change based on various factors, including market conditions and demand. However, we understand your frustration and apologize for any confusion caused.

Regarding the quality of the base, we appreciate your observation. We constantly strive to improve our products and your feedback will be taken into consideration for future enhancements.

We apologize for the issue you encountered with the motor after the warranty expired. Our customer service team is always available to assist you with any concerns you may have. Please feel free to reach out to them for further assistance.

Thank you once again for your valuable feedback. We truly value your loyalty and appreciate your support. If you have any further questions or ne

In [55]:
def get_completion(prompt, model="gpt-3.5-turbo",temperature=0): # Andrew mentioned that the prompt/ completion paradigm is preferable for this class
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [56]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service.
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_completion(prompt, temperature=0.7)
print(response)

Dear valued customer,

Thank you for taking the time to share your review with us. We appreciate your feedback and are sorry to hear about your experience with the pricing and quality of our 17 piece system.

We apologize for any inconvenience caused by the increase in prices during the second week of December. Our aim is to provide fair and competitive pricing to all our customers, and we understand your frustration. If you have any further concerns regarding pricing or any other issues, we encourage you to reach out to our customer service team. They will be more than happy to assist you.

We also apologize for the issues you encountered with the base of the system and the motor making a funny noise after a year of use. We strive to maintain the highest quality standards in our products, and we regret that this was not reflected in your purchase. Your feedback regarding the base and motor will be shared with our product development team for further improvement.

Once again, we apprec

In [57]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service.
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_completion(prompt, temperature=0.0)
print(response)

Dear Valued Customer,

Thank you for taking the time to share your review with us. We appreciate your feedback and apologize for any inconvenience you may have experienced.

We are sorry to hear about the price increase you noticed in December. Our pricing is subject to change based on various factors, including market conditions and demand. We understand that this may have been disappointing for you, and we apologize for any confusion caused.

Regarding the quality of the base, we appreciate your observation. We constantly strive to improve our products, and your feedback will be taken into consideration for future enhancements.

We apologize for the issue you encountered with the motor after a year of use. Our customer service team is here to assist you, even if the warranty has expired. Please feel free to reach out to them at [customer service contact] for further assistance.

We value your loyalty and appreciate your support. If there is anything else we can do to assist you, plea

# ChatBot

In [58]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [59]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)

In [60]:
'''
Note: 1
1. I would like to order pizza, menu, please!
2. Ahh, pepperoni pizza is fine.
3. small one
4. Please add extra cheese and peppers, and ... do you have fried chicken?
5. Could you review my order?
6. A bottle of water, please.
7. I prefer delivered. Is there any additional cost?
8. OK. Howard Street Oswego NY 13126 USA
9. That's it
10. Correct!
11. All done?
12. Did you forgot collect my payment?
'''

"\nNote: 1\n1. I would like to order pizza, menu, please!\n2. Ahh, pepperoni pizza is fine.\n3. small one\n4. Please add extra cheese and peppers, and ... do you have fried chicken?\n5. Could you review my order?\n6. A bottle of water, please.\n7. I prefer delivered. Is there any additional cost?\n8. OK. Howard Street Oswego NY 13126 USA\n9. That's it\n10. Correct!\n11. All done?\n12. Did you forgot collect my payment?\n"

In [61]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

<ipython-input-59-e97edc341c46>:10: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.3, use 'styles' instead.
  pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))


Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

<ipython-input-59-e97edc341c46>:10: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.3, use 'styles' instead.
  pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
